Modify the regression scratch code in our lecture such that:

- Implement early stopping in which if the absolute difference between old loss and new loss does not exceed certain threshold, we abort the learning.

- Implement options for stochastic gradient descent in which we use only one sample for training.  Make sure that sample does not repeat unless all samples are read at least once already.

- Put everything into class.

In [1]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

boston = load_boston()

In [56]:
boston
X = boston.data
y = boston.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

intercept = np.ones((X_train.shape[0], 1))
X_train = np.concatenate((intercept, X_train), axis=1)
intercept = np.ones((X_test.shape[0], 1))
X_test = np.concatenate((intercept, X_test), axis=1)

In [69]:
class RegressionModel:
    
    def __init__(self, max_iteration = 10000, alpha = 0.0001, loss_old = 10000, method="batch",
    tol = 1e-5) -> None:
        self.max_iter = max_iteration
        self.alpha = alpha
        self.loss_old = loss_old
        self.method = method
        self.stop_iter = 0
        self.tol = tol
        self.mini_batch_size = 100

    def closed_form(self, X, y):
        self.theta = np.linalg.inv(X.T @ X) @ X.T @ y
        return self.theta

    def predict(self, X):
        return np.dot(X, self.theta)

    def mse(self, X, y, yhat):
        return ((y - yhat)**2).sum() / X.shape[0]
    
    def h_theta(self, X, theta):
        return X @ theta

    def gradient(self, X, loss):
        return X.T @ loss

    def batch_gradient_descend(self, X, y):
        theta = np.zeros(X.shape[1])
        sto_used_index = []
        for i in range(self.max_iter):
            
            if self.method == "batch":
                X_train = X
                y_train = y
            elif self.method == "mini-batch":
                index = np.random.randint(X.shape[0])
                while index in sto_used_index:
                    index = np.random.randint(X.shape[0])
                X_train = X[index:index + self.mini_batch_size, :]
                y_train = y[index:index + self.mini_batch_size]
                sto_used_index.append(index)
                if len(sto_used_index) == X.shape[0]:
                    sto_used_index = []
            elif self.method == "stochastic":
                index = np.random.randint(X.shape[0])
                while index in sto_used_index:
                    index = np.random.randint(X.shape[0])
                X_train = X[index, :].reshape(1, -1)
                y_train = y[index]
                sto_used_index.append(index)
                if len(sto_used_index) == X.shape[0]:
                    sto_used_index = []

            yhat = self.h_theta(X_train, theta)
            cost = yhat - y_train
            grad = self.gradient(X_train, cost)
            theta = theta - self.alpha * grad
            
            loss_new = self.mse(X_train, y_train, yhat)
            if self.is_early_stop(loss_new, self.loss_old, self.tol):
                self.stop_iter = i
                break
            else:
                self.stop_iter = i
                self.loss_old = loss_new
            
        self.theta = theta
        return theta

    def is_early_stop(self, loss_new, loss_old, tol) -> bool:
        return np.abs(loss_new - loss_old) < tol


In [70]:
model = RegressionModel(method="batch")
theta = model.batch_gradient_descend(X_train, y_train)
yhat = model.predict(X_test)

print("MSE: ", model.mse(X_test, y_test, yhat))
print("Stop at Iteration:", model.stop_iter)

MSE:  17.867060332953567
Stop at Iteration: 1150


In [67]:
model = RegressionModel(method="stochastic")
theta = model.batch_gradient_descend(X_train, y_train)
yhat = model.predict(X_test)

print("MSE: ", model.mse(X_test, y_test, yhat))
print("Stop at Iteration:", model.stop_iter)

MSE:  79.17960964416972
Stop at Iteration: 9999


In [77]:
model = RegressionModel(method="mini-batch", max_iteration=20000)
theta = model.batch_gradient_descend(X_train, y_train)
yhat = model.predict(X_test)

print("MSE: ", model.mse(X_test, y_test, yhat))
print("Stop at Iteration:", model.stop_iter)

MSE:  18.08405967809017
Stop at Iteration: 19999
